In [1]:
import pandas as pd
import os
import json
from collections import defaultdict
os.getcwd()

'D:\\pgkb_graph'

In [2]:
diplotype_folder = "diplotype"

mentioned_genes = ["CYP2B6", "CYP2C9", "CYP2C19", "CYP2D6", "CYP3A5",
                   "DPYD", "NUDT15", "SLCO1B1", "TPMT", "UGT1A1"]

diplotype_path_dict = {g: "" for g in mentioned_genes}
for path in os.listdir(diplotype_folder):
    for gene in mentioned_genes:
        if "{}_Diplotype".format(gene) in path:
            diplotype_path_dict[gene] = os.path.join(diplotype_folder, path)
for key,value in sorted(diplotype_path_dict.items(), key=lambda x: x[0]):
    print(key, value)

CYP2B6 diplotype\CYP2B6_Diplotype_Phenotype_Table.xlsx
CYP2C19 diplotype\CYP2C19_Diplotype_Phenotype_Table.xlsx
CYP2C9 diplotype\CYP2C9_Diplotype_Phenotype_Table.xlsx
CYP2D6 diplotype\CYP2D6_Diplotype_Phenotype_Table.xlsx
CYP3A5 diplotype\CYP3A5_Diplotype_Phenotype_Table.xlsx
DPYD diplotype\DPYD_Diplotype_Phenotype_Table.xlsx
NUDT15 diplotype\NUDT15_Diplotype_Phenotype_Table.xlsx
SLCO1B1 diplotype\SLCO1B1_Diplotype_Phenotype_Table.xlsx
TPMT diplotype\TPMT_Diplotype_Phenotype_Table.xlsx
UGT1A1 diplotype\UGT1A1_Diplotype_Phenotype_Table.xlsx


In [12]:
xl = pd.ExcelFile('diplotype\CYP2B6_Diplotype_Phenotype_Table.xlsx')
print(xl.sheet_names)  # see all sheet names

for sheet in xl.sheet_names:
    if "diplotype" in sheet.lower():
        ### read and handle diplotype
        df_diplotype = pd.read_excel(xl, sheet_name=sheet).fillna("")
        diplotype_column = ""
        phenotype_column = ""
        ehr_notation_column = ""
        score_column = ""
        variant_1_column = ""
        variant_2_column = ""
        
        for column in df_diplotype.columns:
            if "diplotype" in column.lower():
                diplotype_column = column
            elif "phenotype" in column.lower() or "metabolizer" in column.lower():
                phenotype_column = column
            elif "notation" in column.lower():
                ehr_notation_column = column
            elif "score" in column.lower():
                score_column = column
            elif "variant" in column.lower() and "1" in column:
                variant_1_column = column
            elif "variant" in column.lower() and "2" in column:
                variant_2_column = column
        
        diplotype_dict = {}
        for index, row in df_diplotype.iterrows():
            if diplotype_column != "":
                diplotype = row[diplotype_column]
            elif variant_1_column != "" and variant_2_column != "":
                diplotype = "{}/{}".format(row[variant_1_column], row[variant_2_column])
            else:
                continue
                
            diplotype_dict[diplotype] = {
                "phenotype": row[phenotype_column] if phenotype_column != "" else "",
                "ehr_notation": row[ehr_notation_column] if ehr_notation_column != "" else "",
                "score": row[score_column] if score_column != "" else ""
            }
        
            
    elif "consult note" in sheet.lower():
        ### read and handle consult note
        df_consult = pd.read_excel(xl, sheet_name=sheet).fillna("")
        phenotype_column = ""
        ehr_notation_column = ""
        consultation_column = ""
        score_column = ""
        for column in df_consult.columns:
            if "phenotype" in column.lower() or "metabolizer" in column.lower():
                phenotype_column = column
            elif "notation" in column.lower():
                ehr_notation_column = column
            elif "consultation" in column.lower():
                consultation_column = column
            elif "score" in column.lower():
                score_column = column
                
        phenotype_dict = {}
        for index, row in df_consult.iterrows():
            phenotype = row[phenotype_column]
            phenotype_dict[phenotype] = {
                "ehr_notation": row[ehr_notation_column] if ehr_notation_column != "" else "",
                "consultation": row[consultation_column] if consultation_column != "" else  "",
                "score": row[score_column] if score_column != "" else ""
            }
            

['Possible CYP2B6 Diplotype', '2B6 Interpretation Consult Note', 'CYP2B6 Implementation Workflow']


In [14]:
diplotype_dict

{'CYP2B6 Normal Metabolizer ': {'phenotype': '',
  'ehr_notation': 'Normal/Routine/Low Risk',
  'score': ''},
 'Indeterminate': {'phenotype': '', 'ehr_notation': 'None', 'score': ''},
 'CYP2B6 Rapid Metabolizer': {'phenotype': '',
  'ehr_notation': 'Normal/Routine/Low Riskc',
  'score': ''},
 'CYP2B6 Intermediate Metabolizer': {'phenotype': '',
  'ehr_notation': 'Abnormal/Priority/High Risk',
  'score': ''},
 'CYP2B6 Ultrarapid Metabolizer': {'phenotype': '',
  'ehr_notation': 'Normal/Routine/Low Riskc',
  'score': ''},
 'CYP2B6 Poor Metabolizer': {'phenotype': '',
  'ehr_notation': 'Abnormal/Priority/High Risk',
  'score': ''},
 '': {'phenotype': '', 'ehr_notation': '', 'score': ''}}